## Model Training

In [3]:
import pandas as pd

In [5]:
df = pd.read_csv('data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [7]:
df = df.drop(labels = ['id'] , axis = 1)

In [8]:
## Independent and dependent Features
X = df.drop(labels=['price'] , axis = 1)
Y = df[['price']]

In [9]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [26]:
# Define the custom rnking for each ordinal variable
cut_categories = ['Fair' , 'Good' , 'Very Good' , 'Premium' , 'Ideal']
color_categories = ['D' , 'E' , 'F' , 'G' , 'H' , 'I' , 'J']
clarity_categories = ['I1' , 'SI2' , 'SI1' , 'VS2' , 'VS1' , 'VVS2' ,'VVS1' , 'IF']

In [12]:
from sklearn.impute import SimpleImputer ## Handling missing values

In [13]:
from sklearn.preprocessing import StandardScaler #handling feature scaling

In [15]:
from sklearn.preprocessing import OrdinalEncoder # Ordinal encoding

In [16]:
##pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [27]:
##Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer' , SimpleImputer(strategy = 'median')),
        ('scaler' , StandardScaler())
    ]
)

#categorigal pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer' , SimpleImputer(strategy = 'most_frequent')),
        ('Ordinalencoder' , OrdinalEncoder(categories = [cut_categories , color_categories , clarity_categories])),
        ('scaler' , StandardScaler())
    ]                                           
)
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline , numerical_cols),
    ('cat_pipeline' , cat_pipeline, categorical_cols)
])

In [28]:
## Train test split
from sklearn.model_selection import train_test_split

x_train , x_test , y_train , y_test = train_test_split(X, Y , test_size = 0.30 , random_state = 42)

In [33]:
X_train = pd.DataFrame(preprocessor.fit_transform(x_train) , columns = preprocessor.get_feature_names_out())
x_test = pd.DataFrame(preprocessor.transform(x_test) , columns = preprocessor.get_feature_names_out())

In [34]:
X_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,-0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,-0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,1.526720,-0.648127
...,...,...,...,...,...,...,...,...,...
135496,-0.629077,-1.500179,1.964434,-0.546492,-0.518125,-0.644575,-1.137644,-0.936747,-0.648127
135497,2.411307,0.443325,2.485700,1.919078,1.872797,1.930288,-1.137644,-0.320880,-0.648127
135498,0.923460,0.906065,0.400636,0.992240,0.921862,1.047891,-0.131772,0.294987,0.018164
135499,-1.038774,-0.667249,-0.641897,-1.212375,-1.197364,-1.252127,-1.137644,0.294987,2.017037


In [36]:
## model training 
from sklearn.linear_model import LinearRegression , Lasso , Ridge , ElasticNet
from sklearn.metrics import r2_score , mean_absolute_error , mean_squared_error

In [37]:
regression = LinearRegression()
regression.fit(X_train , y_train)

LinearRegression()

In [38]:
regression.coef_

array([[ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
         -494.17005097,   -76.32351645,    68.80035873,  -464.67990411,
          652.10059539]])

In [39]:
regression.intercept_

array([3976.8787389])

In [40]:
import numpy as np 
def evaluate_model(true , predicted):
    mae = mean_absolute_error(true , predicted)
    mse = mean_squared_error(true , predicted)
    rmse = np.sqrt(mean_squared_error(true , predicted))
    r2_square = r2_score(true , predicted)
    return mae , mse , r2_square

In [44]:
## train multiple models

models = {
    'LinearRegression' : LinearRegression() ,
    'Lasso' : Lasso(),
    'Ridge':Ridge(),
    'ElasticNet' : ElasticNet()
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train , y_train)

    #make Predictions
    y_pred = model.predict(x_test)

    mae , mse , r2_square = evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    

    print('Model Training Performance')
    print("RMSE:",mse)
    print("MAE:" , mse)
    print("R2 Score:" , r2_square*100)

    r2_list.append(r2_square)

    print('='*35)
    print('\n')
          


LinearRegression
Model Training Performance
RMSE: 1029473.3531156846
MAE: 1029473.3531156846
R2 Score: 93.62906819996049


Lasso
Model Training Performance
RMSE: 1029533.1506505491
MAE: 1029533.1506505491
R2 Score: 93.62869814082755


Ridge
Model Training Performance
RMSE: 1029482.8101268952
MAE: 1029482.8101268952
R2 Score: 93.62900967491632


ElasticNet
Model Training Performance
RMSE: 2351174.8713978743
MAE: 2351174.8713978743
R2 Score: 85.44967219374031




In [45]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']